In [ ]:
  !pip install language-tool-python


In [ ]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)

PyTorch version: 2.1.0+cu118
CUDA version: 11.8


In [ ]:
!pip install torch_geometric -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install torch_sparse pyg_lib -f https://data.pyg.org/whl/torch-2.1.0+cu118.html


Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html


In [ ]:
import numpy as np
import pandas as pd
import regex as re
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import make_scorer, accuracy_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import language_tool_python
from concurrent.futures import ProcessPoolExecutor
from sklearn.naive_bayes import MultinomialNB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
seed = 202
train = pd.read_csv("/content/drive/MyDrive/train_v2_drcat_02.csv")
external_train = pd.read_csv("/content/drive/MyDrive/train_essays.csv")
external_train.rename(columns={'generated': 'label'}, inplace=True)

In [ ]:
def seed_everything(seed=202):
    import random
    random.seed(seed)
    np.random.seed(seed)

seed_everything(seed)

In [ ]:
tool = language_tool_python.LanguageTool('en-US')

def correct_sentence(sentence):
    return tool.correct(sentence)

def correct_df(df):
    with ProcessPoolExecutor() as executor:
        df['text'] = list(executor.map(correct_sentence, df['text']))

def how_many_typos(text):
    return len(tool.check(text))

In [ ]:
display(train)

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False
...,...,...,...,...,...
44863,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True
44864,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True
44865,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True
44866,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True


In [ ]:
not_persuade_df = train[train['source'] != 'persuade_corpus']
persuade_df = train[train['source'] == 'persuade_corpus']
sampled_persuade_df = persuade_df.sample(n=6000, random_state=42)

In [ ]:
all_human = set(list(''.join(sampled_persuade_df.text.to_list())))
other = set(list(''.join(not_persuade_df.text.to_list())))
chars_to_remove = ''.join([x for x in other if x not in all_human])
print(chars_to_remove)

translation_table = str.maketrans('', '', chars_to_remove)

注□全一😎🚀💡📺手📄⏰🔍🎬🐆👧🌏🐳🏯÷🥦🏥🙊🍜的🍓🥜👦😄🚕う🎮🙏🍰🔥😨🥳📱🏈🌅а😔💔🏙👀🎭🦎É🎶😴🐸🚔🕒是有🎊🥩🤫😓🚨😊📦🎢🕵す💊🍄🕰🛍😱道🥔🌳👬💯🛀👏🍴🚫影🧦😆🛠🐠止🌯🎈🏫🥑ü🌟上👕🧚と🙅💰—😃👮🎸🎯部🍽🌱🚭🍭💻🙋😕安🐶🏃ち🍎ç👩👥🌄📞🚑🐱🥖🌮🍲🌿🤞保🇵🧠🎾🦁🍟在🏀😢完🏔せ✨🥕将👫🌲♂😒❄🤷👻📣зé🍗👍🏛💫ãん🐾🛬集💨🧘🏆📉🏼🍿📹🌨🌽应🎩🛋所意🐦🌴🍮🇫🔜“💜が🐟🥤🎣用力🎨р👪🛸🦄司💚👂🤤😈该🚚​п🌃🧹🙀🥗🌷📷🦐📈🕹😤😠💘💅🌎­🏞🎓🌈📖🚌😜💇🤯🍔🧐🏕🍝者时🧬💦📰机🥟д💀с’😘🧑


In [ ]:
def remove_chars(s):
    return s.translate(translation_table)

In [ ]:
train = pd.concat([train, external_train])
train['text'] = train['text'].apply(remove_chars)
train['text'] = train['text'].str.replace('\n', '')

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/test_essays.csv')
test['text'] = test['text'].str.replace('\n', '')
test['text'] = test['text'].apply(remove_chars)
correct_df(test)

In [ ]:
from sklearn.decomposition import TruncatedSVD


df = pd.concat([train['text'], test['text']], axis=0)
vectorizer = TfidfVectorizer( max_features = 20000,
                             ngram_range=(1, 1), tokenizer=lambda x: re.findall(r'[^\W]+', x), token_pattern=None, strip_accents='unicode')
tfidf_matrix = vectorizer.fit_transform(df)

svd = TruncatedSVD(n_components=64)
reduced_matrix = svd.fit_transform(tfidf_matrix)


In [ ]:
display(test)

,id,prompt_id,text
0,0000aaaa,2,AAA BBB CCC.
1,1111bbbb,3,BBB CCC DDD.
2,2222cccc,4,CCC DDD EEE.


In [ ]:
print(reduced_matrix.shape[1])

64


In [ ]:
import itertools
train['prompt_id'] = train['prompt_name'].factorize()[0]
prompts = train['prompt_id']


edges = []
for prompt in prompts.unique():
    indices = train[train['prompt_id'] == prompt].index
    for i, j in itertools.combinations(indices, 2):
        edges.append([i, j])



In [ ]:
import torch
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()


In [ ]:

from torch_geometric.data import Data

labels_tensor = torch.tensor(train['label'].values, dtype=torch.long)

data = Data(x=torch.tensor(reduced_matrix[:train.shape[0]], dtype=torch.float),
            edge_index=edge_index,
            y=labels_tensor)



In [ ]:
train_mask = np.zeros(len(data.y))
train_mask[0:round(len(data.y)*0.8)] = 1

val_mask = np.ones(len(data.y))
val_mask[0:round(len(data.y)*0.8)] = 0



idx_train = torch.tensor(train_mask).type(torch.bool)
idx_val = torch.tensor(val_mask).type(torch.bool)

data.train_mask = idx_train
data.val_mask = idx_val

In [ ]:
print(data.train_mask, data.val_mask )

tensor([ True,  True,  True,  ..., False, False, False]) tensor([False, False, False,  ...,  True,  True,  True])


In [ ]:
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F

class GraphSAGE(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(num_features, 128)
        self.conv2 = SAGEConv(128, num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.log_softmax(self.conv2(x, edge_index), dim=1)
        return x


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphSAGE(num_features=reduced_matrix.shape[1], num_classes=2).to(device)


In [ ]:
import torch_geometric
from torch_geometric.loader import NodeLoader
from torch_geometric.sampler import NeighborSampler

print(data)
# Crear el sampler
sampler = NeighborSampler(data,  num_neighbors = [200,200])

loader = NodeLoader(data, batch_size=500, shuffle=True, node_sampler=sampler)

# Obtener un batch de nodos
sampled_data = next(iter(loader))
print(sampled_data)

Data(x=[46246, 64], edge_index=[2, 80250536], y=[46246], train_mask=[46246], val_mask=[46246])
Data(x=[41768, 64], edge_index=[2, 5927936], y=[41768], train_mask=[41768], val_mask=[41768], n_id=[41768], e_id=[5927936], num_sampled_nodes=[3], num_sampled_edges=[2], input_id=[500], batch_size=500)


In [ ]:
# from torch_geometric.loader import ClusterLoader
# from torch_geometric.loader import ClusterData

# cluster_data = ClusterData(data, num_parts=5000)
# loader = ClusterLoader(cluster_data, batch_size=1, shuffle=False)

# sampled_data = next(iter(loader))
# print(sampled_data)

In [ ]:
def print_model_stats(model):
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name, "weight:", param.data.norm().item(), "grad:", param.grad.norm().item() if param.grad is not None else None)

In [ ]:
torch.autograd.set_detect_anomaly(True)


In [ ]:
num_epochs = 200
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    # Fase de entrenamiento
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch in loader:
        batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)



        # Aplicar la máscara de entrenamiento
        train_mask = batch.train_mask
        loss = F.nll_loss(out[train_mask], batch.y[train_mask])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Calcular la precisión
        preds = out.max(1)[1]
        correct += preds[train_mask].eq(batch.y[train_mask]).sum().item()
        total += train_mask.sum().item()
        if torch.isnan(loss):
            print("batch.x:", batch.x)
            print("batch.edge_index:", batch.edge_index)
            print("train_mask:", batch.train_mask)
            print("val_mask:", batch.val_mask)
            print(batch.y)
            print(batch.x)
            print(f"NaN detected at epoch {epoch}")
            print_model_stats(model)
            break

    train_accuracy = 100. * correct / total


    print(f"Epoch {epoch}, Loss: {total_loss / len(loader)}, Train Accuracy: {train_accuracy}%")

    # Fase de validación
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for batch in loader:
            batch.to(device)
            out = model(batch.x, batch.edge_index)

            # Aplicar la máscara de validación
            val_mask = batch.val_mask
            val_correct += out[val_mask].max(1)[1].eq(batch.y[val_mask]).sum().item()
            val_total += val_mask.sum().item()

    val_accuracy = 100. * val_correct / val_total
    print(f"Epoch {epoch}, Val Accuracy: {val_accuracy}%")

Epoch 0, Loss: 0.4685219253904076, Train Accuracy: 75.67702935181866%
Epoch 0, Val Accuracy: 62.959994522556194%
Epoch 1, Loss: 0.15249373948061337, Train Accuracy: 96.46049048245604%
Epoch 1, Val Accuracy: 87.35118987464578%
Epoch 2, Loss: 0.06277197086682884, Train Accuracy: 98.57428132743318%
Epoch 2, Val Accuracy: 92.74805833017409%
Epoch 3, Loss: 0.04203489514928992, Train Accuracy: 98.95228405253316%
Epoch 3, Val Accuracy: 95.16119304565997%
Epoch 4, Loss: 0.03324514184828087, Train Accuracy: 99.12770794648733%
Epoch 4, Val Accuracy: 95.98525969621771%
Epoch 5, Loss: 0.028510528907019606, Train Accuracy: 99.23663358205901%
Epoch 5, Val Accuracy: 96.44063203956921%
Epoch 6, Loss: 0.025474931704260968, Train Accuracy: 99.2947926270806%
Epoch 6, Val Accuracy: 96.63594005284709%
Epoch 7, Loss: 0.02319816249592971, Train Accuracy: 99.34305712020756%
Epoch 7, Val Accuracy: 96.81342961079855%
Epoch 8, Loss: 0.02153685428602721, Train Accuracy: 99.37516137816809%
Epoch 8, Val Accuracy: 9